In [46]:
!pip install mne
!pip install pyxdf
!pip install PyWavelets
import pyxdf
import pandas as pd
import seaborn as sns
sns.set_theme()
%load_ext autoreload
%autoreload 2
from utils_clean import show_streams, find_stream, get_duration, get_time_series, set_channel_names, plot_channel_correlation,detect_bad_channels,plot_bads, ica_analysis, ica_plot,remove_breaks,build_class_epochs_mne,build_cat_epochs_mne,create_dataset,return_dataset,plot_topo,plot_eeg,plot_evoked,standartization, labelS02


python(96945) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(96949) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(96951) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
path_file = '/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/EEG_Analysis/P10/P10_02.xdf'
data, header = pyxdf.load_xdf(path_file)
show_streams(data)
name = path_file.split('/')[-1].split('.')[0]
ch_names = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 
                    'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 
                    'P4', 'P8', 'POz', 'O1', 'O2', 'EOG', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 
                    'F6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CP4', 'P5', 'P1', 'P2', 
                    'P6', 'PO5', 'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Oz']

eeg_stream = find_stream('eeg', data)
marker_stream = find_stream('marker', data)
df_marker = get_time_series(marker_stream)
sfreq = float(eeg_stream["info"]["nominal_srate"][0])

Channel 0:
Effective Rate: 0
Start TimeStamp: 
Channel Name: ['EE225-000000-000867-02-TRG-DESKTOP-4OD688D']
Channel Type: ['Markers']
Time Series Shape: (1, 0)
--------------------
Channel 1:
Effective Rate: 511.99999929642524
Start TimeStamp: 
Channel Name: ['EE225-000000-000867-02-DESKTOP-4OD688D']
Channel Type: ['EEG']
Time Series Shape: (1843401, 66)
--------------------
Channel 2:
Effective Rate: 0
Start TimeStamp: 
Channel Name: ['LSLMarkersInletStreamName2']
Channel Type: ['Markers']
Time Series Length: 606
--------------------


In [48]:
# eeg_time_series = raw_reconstructed.get_data()
eeg_time_series = eeg_stream["time_series"]
# eeg_data = eeg_stream["time_series"].T
eeg_timestamps = eeg_stream["time_stamps"] #Timestamps for each sample in the EEG time series.
# timeseries and stamps for all markers
event_time_series= marker_stream['time_series']
event_time_stamps= marker_stream['time_stamps']


## Label the whole Dataset

In [49]:
group_mapping = {
    'banana': 1,
    'strawberry': 2,
    'panda': 3,
    'basketball': 4,
    'face': 5,
    'tiger': 6,
    'bananaText': 1,
    'strawberryText': 2,
    'pandaText': 3,
    'basketballText': 4,
    'faceText': 5,
    'tigerText': 6
}

# Function to extract the group and specific labels from the marker string
def extract_labels(marker):
    parts = marker.split()  # Split the marker string into parts
    
    # Iterate over the parts to find the group name
    for part in parts:
        group_name = ''.join(filter(str.isalpha, part))  # Keep only alphabetic characters
        if group_name in group_mapping:
            group_label = group_mapping[group_name]
            specific_number_str = ''.join(filter(str.isdigit, part))
            specific_label = int(f"{group_label}{specific_number_str}")  # Combine group label with numeric part
            return group_label, specific_label
    return 0, 0  # Default return if no group label is found

# Function to assign trial labels with incremental counts for each occurrence of the same specific label


def assign_trial_label(row):
    trial_counters = {}
    specific_label = row['specific_label_int']
    
    # Increment the trial number every time we see the specific label
    if specific_label in trial_counters:
        trial_counters[specific_label] += 1
    else:
        trial_counters[specific_label] = 1
    
    # Return the trial label with the current trial count
    return int(f"{specific_label}{trial_counters[specific_label]}")
# Apply the function to extract group and specific labels

df_marker[['group_label_int', 'specific_label_int']] = df_marker['marker'].apply(
    lambda x: pd.Series(extract_labels(x)))
print(df_marker[['marker', 'group_label_int', 'specific_label_int']])

# Display the final DataFrame with all relevant columns


                                                         marker  \
16509.686981  Part 1 StimStart basketball1 (UnityEngine.Tran...   
16517.690319  Imagination Start basketball1 (UnityEngine.Tra...   
16525.681583  ImaginationEnd basketball1 (UnityEngine.Transf...   
16529.691365   Part 1 StimStart banana5 (UnityEngine.Transform)   
16537.681920  Imagination Start banana5 (UnityEngine.Transform)   
...                                                         ...   
19323.102760           StimEnd faceText (UnityEngine.Transform)   
19325.093525   Part 3 StimStart panda10 (UnityEngine.Transform)   
19333.099001            StimEnd panda10 (UnityEngine.Transform)   
19335.103789  Part 3 StimStart pandaText (UnityEngine.Transf...   
19343.095370          StimEnd pandaText (UnityEngine.Transform)   

              group_label_int  specific_label_int  
16509.686981                4                  41  
16517.690319                4                  41  
16525.681583                4          

## Label and Epoch Part 1 - Object Presentaiton

In [64]:
import numpy as np
part1_marker = df_marker.iloc[:234]
part1_stim = df_marker[df_marker['marker'].str.startswith('Part 1')].reset_index()
Part1_Stim_onset = part1_marker[part1_marker['marker'].str.startswith('Part 1')].reset_index()
Part1_Stim_onset['numb'] = range(len(Part1_Stim_onset))

dataset_p1_obj = labelS02(8,Part1_Stim_onset,eeg_timestamps,eeg_time_series)

print(part1_stim.shape)
print(dataset_p1_obj[0].shape)




(78, 4)
(798786, 68)


In [65]:
# Assuming 'dataset_p1_obj' is a list of DataFrames, apply the filtering to each DataFrame
df_filtered_list = [
    df[(df['label'] != 900) & (df['group_label'] != 900) & (df['trial_label'] != 900)]
    for df in dataset_p1_obj
]

# Now, 'df_filtered_list' will contain the filtered DataFrames
df_filtered_list
np.save("/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/EEG_Analysis/P10/obj_presentation_label.npy",df_filtered_list)


In [66]:
df_filtered_list[0].shape

(319488, 68)

## Label and Epoch Part1 - Immagination

In [53]:
part1_marker = df_marker.iloc[:234]
part1_imag = df_marker[df_marker['marker'].str.startswith('Imagination Start')].reset_index()
Part1_Imag_onset = part1_marker[part1_marker['marker'].str.startswith('Imagination Start')].reset_index()
Part1_Imag_onset['numb'] = range(len(Part1_Imag_onset))
Part1_Imag_onset

,index,marker,group_label_int,specific_label_int,numb
0,16517.690319,Imagination Start basketball1 (UnityEngine.Tra...,4,41,0
1,16537.681920,Imagination Start banana5 (UnityEngine.Transform),1,15,1
2,16557.688488,Imagination Start strawberry7 (UnityEngine.Tra...,2,27,2
3,16577.680661,Imagination Start strawberry3 (UnityEngine.Tra...,2,23,3
4,16597.686349,Imagination Start strawberry5 (UnityEngine.Tra...,2,25,4
...,...,...,...,...,...
73,17977.714221,Imagination Start basketball3 (UnityEngine.Tra...,4,43,73
74,17997.720323,Imagination Start panda7 (UnityEngine.Transform),3,37,74
75,18017.712399,Imagination Start strawberry10 (UnityEngine.Tr...,2,210,75
76,18037.718043,Imagination Start tiger5 (UnityEngine.Transform),6,65,76


In [54]:
dataset_part1_obj_imag = labelS02(event_time=8, subset_Stim_onset=Part1_Imag_onset, eeg_time_series=eeg_time_series,eeg_timestamps=eeg_timestamps)
dataset_part1_obj_imag

[                time  label  group_label  trial_label       Fp1       Fpz  \
 0       16517.690584     41            4          411 -0.008954 -0.013541   
 1       16517.692537     41            4          411 -0.008959 -0.013557   
 2       16517.694490     41            4          411 -0.008970 -0.013588   
 3       16517.696443     41            4          411 -0.008974 -0.013596   
 4       16517.698396     41            4          411 -0.008971 -0.013578   
 ...              ...    ...          ...          ...       ...       ...   
 802873  18085.801914    900          900          900       NaN       NaN   
 802874  18085.803867    900          900          900       NaN       NaN   
 802875  18085.805820    900          900          900       NaN       NaN   
 802876  18085.807773    900          900          900       NaN       NaN   
 802877  18085.809727    900          900          900       NaN       NaN   
 
              Fp2        F7        F3        Fz  ...       PO3

## Label and Epoch Part 2 - Text Stimulus Presentation

In [55]:
part2 = df_marker.iloc[234:450]
part2

,marker,group_label_int,specific_label_int
18098.809468,Part 2 StimStart faceText (UnityEngine.Transform),5,5
18100.814468,Imagination Start faceText (UnityEngine.Transf...,5,5
18102.819238,ImaginationEnd faceText (UnityEngine.Transform),5,5
18104.810192,Part 2 StimStart pandaText (UnityEngine.Transf...,3,3
18106.815094,Imagination Start pandaText (UnityEngine.Trans...,3,3
...,...,...,...
18520.820554,Imagination Start bananaText (UnityEngine.Tran...,1,1
18522.811474,ImaginationEnd bananaText (UnityEngine.Transform),1,1
18524.816316,Part 2 StimStart strawberryText (UnityEngine.T...,2,2
18526.821032,Imagination Start strawberryText (UnityEngine....,2,2


In [56]:
part2_text = part2[part2['marker'].str.startswith('Part 2')].reset_index()
part2_text = part2_text[part2_text['marker'].str.startswith('Part 2')].reset_index()
part2_text['numb'] = range(len(part2_text))
part2_text

,level_0,index,marker,group_label_int,specific_label_int,numb
0,0,18098.809468,Part 2 StimStart faceText (UnityEngine.Transform),5,5,0
1,1,18104.810192,Part 2 StimStart pandaText (UnityEngine.Transf...,3,3,1
2,2,18110.810717,Part 2 StimStart bananaText (UnityEngine.Trans...,1,1,2
3,3,18116.811123,Part 2 StimStart bananaText (UnityEngine.Trans...,1,1,3
4,4,18122.811476,Part 2 StimStart strawberryText (UnityEngine.T...,2,2,4
...,...,...,...,...,...,...
67,67,18500.814161,Part 2 StimStart strawberryText (UnityEngine.T...,2,2,67
68,68,18506.814633,Part 2 StimStart basketballText (UnityEngine.T...,4,4,68
69,69,18512.815284,Part 2 StimStart bananaText (UnityEngine.Trans...,1,1,69
70,70,18518.815579,Part 2 StimStart bananaText (UnityEngine.Trans...,1,1,70


In [57]:
dataset_p2_text = labelS02(2,part2_text,eeg_timestamps,eeg_time_series)
dataset_p2_text

[                 time  label  group_label  trial_label       Fp1       Fpz  \
 0        18098.809727      5            5           51 -0.013302 -0.018405   
 1        18098.811680      5            5           51 -0.013304 -0.018406   
 2        18098.813633      5            5           51 -0.013306 -0.018401   
 3        18098.815586      5            5           51 -0.013306 -0.018396   
 4        18098.817539      5            5           51 -0.013302 -0.018387   
 ...               ...    ...          ...          ...       ...       ...   
 1023572  20097.973792    900          900          900       NaN       NaN   
 1023573  20097.975745    900          900          900       NaN       NaN   
 1023574  20097.977698    900          900          900       NaN       NaN   
 1023575  20097.979651    900          900          900       NaN       NaN   
 1023576  20097.981604    900          900          900       NaN       NaN   
 
               Fp2        F7        F3        Fz  

## Label and Epoch Part 2 - Text Imagination

In [58]:
part2_text_imag = part2[part2['marker'].str.startswith('Imagination Start')].reset_index()
part2_text_imag = part2_text_imag[part2_text_imag['marker'].str.startswith('Imagination Start')].reset_index()
part2_text_imag['numb'] = range(len(part2_text_imag))
part2_text_imag

,level_0,index,marker,group_label_int,specific_label_int,numb
0,0,18100.814468,Imagination Start faceText (UnityEngine.Transf...,5,5,0
1,1,18106.815094,Imagination Start pandaText (UnityEngine.Trans...,3,3,1
2,2,18112.815596,Imagination Start bananaText (UnityEngine.Tran...,1,1,2
3,3,18118.815959,Imagination Start bananaText (UnityEngine.Tran...,1,1,3
4,4,18124.816441,Imagination Start strawberryText (UnityEngine....,2,2,4
...,...,...,...,...,...,...
67,67,18502.819098,Imagination Start strawberryText (UnityEngine....,2,2,67
68,68,18508.819337,Imagination Start basketballText (UnityEngine....,4,4,68
69,69,18514.820000,Imagination Start bananaText (UnityEngine.Tran...,1,1,69
70,70,18520.820554,Imagination Start bananaText (UnityEngine.Tran...,1,1,70


In [59]:
dataset_part2_text_imag = labelS02(2,part2_text_imag,eeg_timestamps, eeg_time_series)
dataset_part2_text_imag

[                 time  label  group_label  trial_label       Fp1       Fpz  \
 0        18100.813633      5            5           51 -0.013272 -0.018476   
 1        18100.815586      5            5           51 -0.013278 -0.018476   
 2        18100.817539      5            5           51 -0.013278 -0.018462   
 3        18100.819492      5            5           51 -0.013260 -0.018436   
 4        18100.821445      5            5           51 -0.013252 -0.018422   
 ...               ...    ...          ...          ...       ...       ...   
 1022546  20097.973792    900          900          900       NaN       NaN   
 1022547  20097.975745    900          900          900       NaN       NaN   
 1022548  20097.977698    900          900          900       NaN       NaN   
 1022549  20097.979651    900          900          900       NaN       NaN   
 1022550  20097.981604    900          900          900       NaN       NaN   
 
               Fp2        F7        F3        Fz  

## Label and Epoch Part 3 -  Object and Text Imagery

In [60]:
#156 total trials - 78 for object and text imagery 
part3 = df_marker.iloc[450:]
print(part3)

                                                         marker  \
18565.092442    Part 3 StimStart tiger7 (UnityEngine.Transform)   
18573.098256             StimEnd tiger7 (UnityEngine.Transform)   
18575.103175  Part 3 StimStart tigerText (UnityEngine.Transf...   
18583.094190          StimEnd tigerText (UnityEngine.Transform)   
18585.099144    Part 3 StimStart panda6 (UnityEngine.Transform)   
...                                                         ...   
19323.102760           StimEnd faceText (UnityEngine.Transform)   
19325.093525   Part 3 StimStart panda10 (UnityEngine.Transform)   
19333.099001            StimEnd panda10 (UnityEngine.Transform)   
19335.103789  Part 3 StimStart pandaText (UnityEngine.Transf...   
19343.095370          StimEnd pandaText (UnityEngine.Transform)   

              group_label_int  specific_label_int  
18565.092442                6                  67  
18573.098256                6                  67  
18575.103175                6          

In [61]:
# Filter for 'Part 3 StimStart' markers
part3_stim = part3[part3['marker'].str.startswith('Part 3 StimStart')].reset_index()

# Use regex to find markers that have "Text" at the end of a word
part3_text = part3_stim[part3_stim['marker'].str.contains(r'\b\w+Text\b')].reset_index()
part3_text['numb'] = range(len(part3_text))
## find markers for object 
part3_obj = part3_stim[~part3_stim['marker'].str.contains(r'\b\w+Text\b')].reset_index()
part3_obj['numb'] = range(len(part3_obj))


In [62]:
part3_text_stim=part3_text.iloc[:39]
part3_text_imag=part3_text.iloc[39:]

In [63]:
dataset_p3_text_stim= labelS02(2,part3_text_stim,eeg_timestamps,eeg_time_series)
dataset_p3_text_imag=labelS02(2,part3_text_imag,eeg_timestamps, eeg_time_series)

print(dataset_p3_text_stim)
print(dataset_p3_text_imag)

/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/Preprocessing/utils_clean.py:586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_Stim_onset['trial_label'] = subset_Stim_onset.apply(assign_trial_label, axis=1)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
part3_obj_stim=part3_obj.iloc[:39]
part3_obj_imag=part3_obj.iloc[39:]

In [ ]:
dataset_p3_obj_stim= labelS02(8,part3_obj_stim,eeg_timestamps,eeg_time_series)
dataset_p3_obj_imag=labelS02(8,part3_obj_imag,eeg_timestamps, eeg_time_series)

print(dataset_p3_obj_stim)
print(dataset_p3_obj_imag)

/Users/arnavkapur/Desktop/Analysis_Imagery/imagery2024/EEG_Analysis/helperfct_S02.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_Stim_onset['trial_label'] = subset_Stim_onset.apply(assign_trial_label, axis=1)
/Users/arnavkapur/Desktop/Analysis_Imagery/imagery2024/EEG_Analysis/helperfct_S02.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_Stim_onset['trial_label'] = subset_Stim_onset.apply(assign_trial_label, axis=1)


[                 time  label  group_label  trial_label       Fp1       Fpz  \
0       320214.001730     33            3          331 -0.011133 -0.024490   
1       320214.003683     33            3          331 -0.011134 -0.024493   
2       320214.005636     33            3          331 -0.011129 -0.024491   
3       320214.007589     33            3          331 -0.011130 -0.024491   
4       320214.009543     33            3          331 -0.011133 -0.024489   
...               ...    ...          ...          ...       ...       ...   
389116  320973.993919    900          900          900 -0.013543 -0.022582   
389117  320973.995872    900          900          900 -0.013549 -0.022582   
389118  320973.997825    900          900          900 -0.013538 -0.022571   
389119  320973.999778    900          900          900 -0.013533 -0.022566   
389120  320974.001731    900          900          900 -0.013535 -0.022566   

             Fp2        F7        F3        Fz  ...       PO3 